ce notebook présente différentes manières de pricer des call ou des put européens ou asiatique
commencons par le plus basique,
#les options européennes

on peut utiliser la méthode de l'arbre binomiale ou bien le Modèle de Black-Scholes commencons par la méthode binomiale qui consiste à supposer qu'a chaque pas de temps notre arbre à une proba $p$ de monter et $1-p$ de descendre,  $p$ dépendant des facteurs de hausse $u$ et du facteur de baisse $d=1/u$

In [23]:
import numpy as np

def prix_option_europeenne_binomiale(S0, K, r, T, sigma, N, option_type="call"):
    """
    Calcule le prix d'une option européenne (call ou put) en utilisant un modèle d'arbre binomial.

    Paramètres :
    S0 : float spot
    K : float strike
    T : float maturité
    r : float Taux d'intérêt sans risque.
    sigma : float Volatilité  sous-jacent.
    N : int nombre de pas modèle
    option_type : str type d'option : "call" ou "put"

    Retourne :
    float Prix de l'option européenne.
    """
    # Calcul des paramètres de l'arbre binomial
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))  # Facteur de hausse
    d = 1 / u                        # Facteur de baisse
    p = (np.exp(r * dt) - d) / (u - d)  # Probabilité neutre au risque

    # Initialisation des matrices pour les prix de l'actif et les valeurs de l'option
    prix = np.zeros((N + 1, N + 1))
    valeur_option = np.zeros((N + 1, N + 1))

    # Calcul des prix possibles de l'actif sous-jacent à l'échéance
    for i in range(N + 1):
        for j in range(i + 1):
            prix[j, i] = S0 * (u ** (i - j)) * (d ** j)

    # Calcul des valeurs de l'option à l'échéance
    for j in range(N + 1):
        if option_type == "call":
            valeur_option[j, N] = max(0, prix[j, N] - K)
        elif option_type == "put":
            valeur_option[j, N] = max(0, K - prix[j, N])

    # Rembobinage de l'arbre pour calculer la valeur de l'option au temps initial
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            valeur_option[j, i] = np.exp(-r * dt) * (p * valeur_option[j, i + 1] + (1 - p) * valeur_option[j + 1, i + 1])

    return valeur_option[0, 0] #le prix de l'option est le noeud initiale

# Exemple
S0 = 105  # spot
K = 100   # strike
r = 0.05  # Taux d'intérêt sans risque
T = 1.0   # maturité
sigma = 0.2  # Volatilité
N = 50    # Nombre de pas de temps

prix_call = prix_option_europeenne_binomiale(S0, K, r, T, sigma, N, option_type="call")
prix_put = prix_option_europeenne_binomiale(S0, K, r, T, sigma, N, option_type="put")

print(f"Prix de l'option call européenne : {prix_call:.2f}")
print(f"Prix de l'option put européenne : {prix_put:.2f}")


Prix de l'option call européenne : 13.86
Prix de l'option put européenne : 3.98


à savoir que pour pricer une option américaine il suffit que faire l'étape de calcul de la valeur de l'option mais à chaque noeuds, passons ensuite au modèle de Black-Scholes dont les solutions sont données après résolutions de l'EDS des prix

In [24]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calcule le prix d'une option européenne selon le modèle de Black-Scholes.
    Paramètres :
    S : float spot
    K : float strike
    T : float maturité
    r : float Taux d'intérêt sans risque.
    sigma : float Volatilité  sous-jacent.
    option_type : str type d'option : "call" ou "put"

    Retourne :
    float Prix de l'option européenne.
    """
    # Calcul des paramètres d1 et d2 pour black-scholes
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":

        prix = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        # Prix de l'option de vente
        prix = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return prix

# Exemple
S0 = 105  # spot
K = 100   # strike
T = 1.0     # maturité
r = 0.05  # Taux d'intérêt sans risque
sigma = 0.2  # Volatilité

# Calcul du prix de l'option d'achat (call)
call_price = black_scholes(S0, K, T, r, sigma, option_type="call")
print(f"Prix de l'option d'achat (call) : {call_price:.2f} €")

# Calcul du prix de l'option de vente (put)
put_price = black_scholes(S0, K, T, r, sigma, option_type="put")
print(f"Prix de l'option de vente (put) : {put_price:.2f} €")


Prix de l'option d'achat (call) : 13.86 €
Prix de l'option de vente (put) : 3.98 €


#2.option asiatique
Une option asiatique est un type d'option dont le payoff dépend de la moyenne (arithmétique ou géométrique) des prix du sous-jacent sur une période donnée, plutôt que du prix observé à un moment unique (comme à l'échéance pour une option européenne classique). Cette caractéristique permet de lisser les fluctuations du marché et de réduire l'impact d'éventuels pics de volatilité sur le prix final de l'option.
###2.1) option asiatique à moyenne géomtrique

Pour une option asiatique à moyenne géométrique en temps continu (échantillonnage continu sur [0, T]), la formule fermée s'écrit généralement de la manière suivante pour une option call:

$$
C = e^{-rT}\left[S_0\, e^{\mu_G + \frac{1}{2}\sigma_G^2}\, N(d_1) - K\, N(d_2)\right]
$$

avec
$\sigma_G = \sigma \sqrt{\frac{T}{3}}\$

$d_1 = \frac{\ln(S_0/K) + \mu_G + \frac{1}{2}\sigma_G^2}{\sigma_G}$
et $d_2 = d_1 - \sigma_G$.

Pour une option put, le prix s'obtient en appliquant le principe de parité put-call adaptée ou en modifiant directement les payoffs.

Voici le code correspondant:





In [18]:
import numpy as np
from scipy.stats import norm

def option_geometrique_asiatique_closed(S0, K, r, sigma, T, option_type="call"):
    """
    Calcule le prix d'une option asiatique à moyenne géométrique par formule fermée.
    Paramètres:
        S0: float Spot.
        K: float  Strike
        r: float  Taux sans risque.
        sigma: float  Volatilité.
        T: float - Maturité .
        option_type: str "call" ou "put".

    Retourne:
        float - Prix de l'option.
    """
    # Calcul des paramètres ajustés pour la moyenne géométrique
    sigma_G = sigma * np.sqrt(T / 3)
    mu_G = 0.5 * (r - 0.5 * sigma**2) * T + (sigma**2 * T) / 6

    # Calcul des variables d1 et d2
    d1 = (np.log(S0 / K) + mu_G + 0.5 * sigma_G**2) / sigma_G
    d2 = d1 - sigma_G

    if option_type == "call":
        prix = np.exp(-r * T) * (S0 * np.exp(mu_G + 0.5 * sigma_G**2) * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == "put":
        prix = np.exp(-r * T) * (K * norm.cdf(-d2) - S0 * np.exp(mu_G + 0.5 * sigma_G**2) * norm.cdf(-d1))
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")
    return prix

### Méthode par Simulation Monte Carlo

Dans la méthode Monte Carlo, on simule de nombreux chemins du sous-jacent suivant la dynamique Black-Scholes. Pour chaque chemin, on calcule la moyenne géométrique (en prenant la moyenne des logarithmes pour éviter des problèmes numériques) et on en déduit le payoff La valeur de l'option est ensuite obtenue par l'actualisation de la moyenne de ces payoffs.

Voici le code associé :

In [17]:
def option_geometrique_asiatique_monte_carlo(S0, K, r, sigma, T, pas=252, num_trajectoire=1000000, option_type="call"):
    """
    Calcule le prix d'une option asiatique à moyenne géométrique par simulation Monte Carlo.

    Paramètres:
        S0: float - Prix initial du sous-jacent.
        K: float - Prix d'exercice.
        r: float - Taux sans risque.
        sigma: float - Volatilité.
        T: float - Maturité (en années).
        pas: int - Nombre de pas dans la simulation temporelle. 252 ici pour prendre le nombre de jour ouvert
        num_trajectoire: int - Nombre de chemins simulés.
        option_type: str - "call" ou "put".

    Retourne:
        float - Prix de l'option.
    """
    dt = T / pas
    # Simulation des trajectoires du sous-jacent
    S = np.zeros((num_trajectoire, pas))
    S[:, 0] = S0
    for i in range(1, pas):
        z = np.random.standard_normal(num_trajectoire)
        S[:, i] = S[:, i-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

    # Calcul de la moyenne géométrique pour chaque trajectoire
    # On utilise la moyenne des logarithmes pour éviter les problèmes numériques
    logS = np.log(S)
    geo_moy = np.exp(np.mean(logS, axis=1))

    # Calcul des payoffs en fonction du type d'option
    if option_type == "call":
        payoffs = np.maximum(geo_moy - K, 0)
    elif option_type == "put":
        payoffs = np.maximum(K - geo_moy, 0)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")

    # Actualisation de la moyenne des payoffs
    prix = np.exp(-r * T) * np.mean(payoffs)
    return prix

In [20]:
# Paramètres d'exemple
S0 = 100.0   # Prix initial
K = 100.0    # Prix d'exercice
r = 0.05     # Taux sans risque
sigma = 0.3  # Volatilité
T = 1.0      # Maturité en années

# Option Call
price_closed_call = option_geometrique_asiatique_closed(S0, K, r, sigma, T, option_type="call")
price_mc_call = option_geometrique_asiatique_monte_carlo(S0, K, r, sigma, T, pas=252, num_trajectoire=1000000, option_type="call")

print("Option asiatique géométrique Call:")
print("Prix en formule fermée: {:.4f}".format(price_closed_call))
print("Prix par Monte Carlo: {:.4f}".format(price_mc_call))

# Option Put
price_closed_put = option_geometrique_asiatique_closed(S0, K, r, sigma, T, option_type="put")
price_mc_put = option_geometrique_asiatique_monte_carlo(S0, K, r, sigma, T, pas=252, num_trajectoire=1000000, option_type="put")

print("\nOption asiatique géométrique Put:")
print("Prix en formule fermée: {:.4f}".format(price_closed_put))
print("Prix par Monte Carlo: {:.4f}".format(price_mc_put))

Option asiatique géométrique Call:
Prix en formule fermée: 8.3363
Prix par Monte Carlo: 7.4797

Option asiatique géométrique Put:
Prix en formule fermée: 5.1940
Prix par Monte Carlo: 5.8043


la différence de prix dans nos modèles peut provenir de plusieurs choses: soit des nombres de chemins et de la discrétisations employés
ou bien du modèles d'évolutions de cours choisit

---

### Remarques

- **Méthode en formule fermée :**  
  Cette méthode est très rapide et fournit une solution exacte dans le cadre de l'hypothèse de moyenne géométrique. Elle repose sur le fait que la moyenne géométrique d'un processus lognormal reste lognormal, ce qui permet de dériver une solution fermée.

- **Méthode Monte Carlo :**  
  Cette approche, bien que plus lente, est très flexible et peut être adaptée à des options plus complexes ou à des cas où la formule fermée n'est pas disponible (par exemple pour les options à moyenne arithmétique). Pour l'option géométrique, elle permet de vérifier la cohérence avec la solution analytique.

Ces deux méthodes vous offrent donc deux perspectives complémentaires pour le pricing d'une option asiatique à moyenne géométrique. pour une moyenne arithmétique on peut procéder comme suivant  avec la méthode de monte-carlo qui est plus simple à implémenter bien que plus longue à compiler

In [21]:
import numpy as np

def asian_option_mc(S0, K, T, r, sigma, pas, num_trajectoire=100000, option_type='call'):
    """
    Prix une option asiatique à moyenne arithmétique par simulation Monte Carlo.

    Paramètres:
        S0 (float): Prix initial du sous-jacent
        K (float): Prix d'exercice
        T (float): Temps jusqu'à maturité (en années)
        r (float): Taux sans risque
        sigma (float): Volatilité
        pas (int): Nombre de pas de temps
        num_trajectoire (int): Nombre de simulations
        option_type (str): 'call' ou 'put'

    retourne:
        float: Prix de l'option
    """

    dt = T / pas
    discount = np.exp(-r * T)
    # Simulation des trajectoires du sous-jacent
    S = np.zeros((num_trajectoire, pas))
    S[:, 0] = S0
    for i in range(1, pas):
        z = np.random.standard_normal(num_trajectoire)
        S[:, i] = S[:, i-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)


    # Calcul de la moyenne arithmétique
    avg_price = np.mean(S[:, 1:], axis=1)

    # Calcul du payoff
    if option_type == "call":
        payoff = np.maximum(avg_price - K, 0)
    elif option_type=="put":
        payoff = np.maximum(K - avg_price, 0)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")

    # Prix de l'option
    option_price = discount * np.mean(payoff)

    return option_price

# Exemple d'utilisation
S0 = 100     # Prix spot
K = 100      # Strike
T = 1        # Maturité (années)
r = 0.05     # Taux sans risque
sigma = 0.2  # Volatilité
n_steps = 252  # Pas de temps (ex: jours ouvrés)
n_sims = 100000 # Nombre de simulations

price_call = asian_option_mc(S0, K, T, r, sigma, n_steps, n_sims, 'call')
print(f"Prix de l'option asiatique (Call): {price_call:.2f}")
price_put = asian_option_mc(S0, K, T, r, sigma, n_steps, n_sims, 'put')
print(f"Prix de l'option asiatique (put): {price_put:.2f}")



Prix de l'option asiatique (Call): 5.72
Prix de l'option asiatique (put): 3.35


comme il n'y a pas de solutions dans le cadre de la moyenne arithmétique on peut faire comme au début pour un call/put européens et utiliser un arbre binomiale un peu plus complexe puisqu'il faut désormais stocker les moyennes et la valeur de l'option

In [22]:
import numpy as np

def binomial_asian(S0, K, T, r, sigma, n, m=50, option_type='call'):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    q = (np.exp(r * dt) - d) / (u - d)

    # Construction de l'arbre du prix du sous-jacent :
    # stock[i, j] correspond au prix du sous-jacent après i pas, avec j mouvements "up".
    stock = np.zeros((n+1, n+1))
    for i in range(n+1):
        for j in range(i+1):
            stock[i,j] = S0 * (u**j) * (d**(i-j))

    # Initialisation des dictionnaires pour stocker la grille de moyennes et la valeur de l'option
    avg_grid = {}  # Contiendra pour chaque noeud (i,j) une grille discrétisée des valeurs de la moyenne arithmétique
    option = {}    # Contiendra pour chaque noeud (i,j) la valeur de l'option associée à chaque valeur discrétisée de la moyenne

    # Étape initiale (t = 0) : au départ la moyenne est simplement le prix initial S0
    avg_grid[(0,0)] = np.array([S0])
    option[(0,0)] = np.zeros(1)

    # Discrétisation du noeud initial : ici la grille est trivialement un unique point
    avg_min = avg_grid[(0,0)][0]
    avg_max = avg_grid[(0,0)][0]
    avg_grid[(0,0)] = np.linspace(avg_min, avg_max, m)
    option[(0,0)] = np.zeros(m)

    # Construction des grilles de moyennes (forward) pour chaque noeud de l'arbre binomial
    for i in range(n):
        for j in range(i+1):
            if (i,j) not in avg_grid:
                continue
            current_avg = avg_grid[(i,j)]
            current_S = stock[i,j]

            # Pour chaque valeur de moyenne dans la grille du noeud courant, on calcule les nouvelles moyennes
            # obtenues après un mouvement "up" et après un mouvement "down"
            for k in range(len(current_avg)):
                avg = current_avg[k]

                # Mouvement up
                next_i, next_j = i+1, j+1
                next_S = current_S * u
                next_avg = (avg * (i+1) + next_S) / (i + 2)
                if (next_i, next_j) not in avg_grid:
                    avg_grid[(next_i, next_j)] = []
                avg_grid[(next_i, next_j)].append(next_avg)

                # Mouvement down
                next_i, next_j = i+1, j
                next_S = current_S * d
                next_avg = (avg * (i+1) + next_S) / (i + 2)
                if (next_i, next_j) not in avg_grid:
                    avg_grid[(next_i, next_j)] = []
                avg_grid[(next_i, next_j)].append(next_avg)

        # À chaque étape, pour chaque noeud atteint, on discrétise la grille des moyennes obtenues
        # afin de limiter le nombre de valeurs et de rendre le problème numérique plus gérable.
        for j in range(i+2):
            key = (i+1, j)
            if key not in avg_grid or len(avg_grid[key]) == 0:
                continue
            avg_min = min(avg_grid[key])
            avg_max = max(avg_grid[key])
            avg_grid[key] = np.linspace(avg_min, avg_max, m)
            option[key] = np.zeros(m)

    # Calcul des payoffs à la maturité (t = T)
    # Pour chaque noeud terminal (niveau n), le payoff est calculé en fonction de la moyenne finale :
    # pour un call, max(moyenne - K, 0) et pour un put, max(K - moyenne, 0)
    for j in range(n+1):
        key = (n, j)
        if key not in option:
            continue
        for k in range(m):
            avg = avg_grid[key][k]
            if option_type == 'call':
                option[key][k] = max(avg - K, 0)
            else:
                option[key][k] = max(K - avg, 0)

    # Rétro-propagation (backward induction) pour calculer la valeur de l'option
    # On part de la maturité et on recule dans l'arbre en actualisant les valeurs
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            key = (i, j)
            if key not in option:
                continue
            current_S = stock[i,j]
            for k in range(m):
                avg = avg_grid[key][k]

                # Calcul de la moyenne pour le mouvement up
                next_avg_up = (avg * (i+1) + current_S * u) / (i + 2)
                key_up = (i+1, j+1)
                if key_up in option:
                    # On utilise l'interpolation linéaire pour obtenir la valeur correspondant à next_avg_up
                    value_up = np.interp(next_avg_up, avg_grid[key_up], option[key_up])
                else:
                    value_up = 0

                # Calcul de la moyenne pour le mouvement down
                next_avg_down = (avg * (i+1) + current_S * d) / (i + 2)
                key_down = (i+1, j)
                if key_down in option:
                    value_down = np.interp(next_avg_down, avg_grid[key_down], option[key_down])
                else:
                    value_down = 0

                # Actualisation de la valeur de l'option
                option[key][k] = np.exp(-r * dt) * (q * value_up + (1 - q) * value_down)

    return option[(0,0)][0]

# Exemple d'utilisation
S0 = 100   # Prix spot
K = 100    # Strike
T = 1      # Maturité (en années)
r = 0.05   # Taux sans risque
sigma = 0.2# Volatilité
n = 50     # Nombre de pas dans l'arbre binomial
m = 50     # Nombre de points dans la discrétisation de la moyenne


prix_call = binomial_asian(S0, K, T, r, sigma, n, m, 'call')
print(f"Prix de l'option asiatique (Call) : {prix_call:.2f}")
prix_put = binomial_asian(S0, K, T, r, sigma, n, m, 'put')
print(f"Prix de l'option asiatique (put) : {prix_put:.2f}")

Prix de l'option asiatique (Call) : 5.80
Prix de l'option asiatique (put) : 3.39
